### Overfitting, underfitting
Переобучение недообучение

In [1]:
# Загрузка необходимых библиотек

import numpy as np #для матричных вычислений
import pandas as pd #для обработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import metrics #метрики
from sklearn import model_selection #методы разделения и валидации
from sklearn import linear_model #линейные модели
from sklearn import tree #деревья решений

from imblearn.over_sampling import SMOTE #алгоритм сэмплирования

# Устанавливаем стиль визуализаций в matplotlib
plt.style.use('seaborn-v0_8')
%matplotlib inline

In [ ]:
#Первичный анализ дата-сета
water_data = pd.read_csv('data/water_potability.csv')
water_data.head()

#Заполним пропуски медаиной в зависимост от класса воды
water_data['ph'] = water_data['ph'].fillna(water_data.groupby('Potability')['ph'].transform('median'))
water_data['Sulfate'] = water_data['Sulfate'].fillna(water_data.groupby('Potability')['Sulfate'].transform('median'))
water_data['Trihalomethanes'] = water_data['Trihalomethanes'].fillna(water_data.groupby('Potability')['Trihalomethanes'].transform('median'))

# Разделим выборку на набор данных X и вектор ответов y
X = water_data.drop('Potability', axis=1)
y = water_data['Potability']

# Разобъем датасет на тренировочную и тестовую (валидационную) выборки
X_train, X_valid, y_train, y_valid = model_selection.train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print(f'Train shape: {X_train.shape}')
print(f'Test shape: {X_valid.shape}')

Train shape: (2620, 9)
Test shape: (656, 9)


In [18]:
#Создаем модель
model_bad = tree.DecisionTreeClassifier(
    criterion='entropy',
    class_weight='balanced', #балансировка классов
    random_state=42
)

#Обучаем модель на тренирововчной выборке
model_bad.fit(X_train, y_train)
#Деалем предсказания для тренировочной и валидационной выборок
y_train_pred = model_bad.predict(X_train)
y_valid_pred = model_bad.predict(X_valid)
#Вычисляем значение метрик для выборок
print('-'*40)
print(f'{" " * 10}Train F1-score {metrics.f1_score(y_train, y_train_pred):.2}')
print(f'{" " * 10}Valid F1-score {metrics.f1_score(y_valid, y_valid_pred):.2}')
print('-'*40)
print('Модель переучена')

----------------------------------------
          Train F1-score 1.0
          Valid F1-score 0.67
----------------------------------------
Модель переучена


Проверим предположение о переобучении модели на кросс-валидации K-Fold

In [16]:
# Создаем объект кроссвалидатор k-fold со стратификацией
skf = model_selection.StratifiedKFold(n_splits=5)

# Считаем метрики на кросс-валидации со стратификацией
cv_metrics = model_selection.cross_validate(
    estimator=model_bad, #модель
    X=X,
    y=y,
    cv=skf, # кросс-валидатор
    scoring='f1', #метрика
    return_train_score=True #Считаем метрику и на тренировочных фолдах
)
display(cv_metrics) # показываем метрику на всех пяти фолдах
print('Видно что на тренировочной выборке метрика = 1, сигнал о переобучении модели.')

{'fit_time': array([0.09674001, 0.08977365, 0.04986715, 0.07775211, 0.07530785]),
 'score_time': array([0.01695442, 0.00497198, 0.00395751, 0.00598478, 0.00498819]),
 'test_score': array([0.61445783, 0.68421053, 0.62332696, 0.63276836, 0.70119522]),
 'train_score': array([1., 1., 1., 1., 1.])}

Видно что на тренировочной выборке метрика = 1, сигнал о переобучении модели.


In [17]:
# подсчитаем среднее значение F1-score на выборках
print('Train k-fold mean f1: {:.2f}'.format(np.mean(cv_metrics['train_score'])))
print('Valid k-fold mean f1: {:.2f}'.format(np.mean(cv_metrics['test_score'])))

Train k-fold mean f1: 1.00
Valid k-fold mean f1: 0.65
